# Dual UR Example

In this notebook we give an example of a region certified for a 12-DOF dual, armed robot. The optimization program for this example is very large and cannot be run on the RAM available on Deepnote. Therefore, we provide a saved example region and demonstrate the range of postures achievable in this region.

This notebook runs in a handful of seconds

In [ ]:
%load_ext autoreload

In [ ]:
import numpy as np
from functools import partial
import ipywidgets as widgets
from IPython.display import display

In [ ]:
#pydrake imports
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptions, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions
from pydrake.all import PiecewisePolynomial
import time


In [ ]:
from ur3e_demo import UrDiagram, SetDiffuse
import ur3e_demo
import visualization_utils as viz_utils

# Build and set up the visualization the plant

Click on the link at the bottom to view a visualization of the plant.

In [ ]:
bilinear_alt_result_file = "dual_ur3e_bil_alt_result.npz"
load_data = bilinear_alt_result_file
weld_wrist = False
with_gripper = True

In [ ]:
ur = UrDiagram(num_ur = 2, 
                weld_wrist = weld_wrist,
                 add_shelf = False,
                 add_gripper = with_gripper)
diagram_context = ur.diagram.CreateDefaultContext()
plant_context = ur.plant.GetMyMutableContextFromRoot(
    diagram_context)
scene_graph_context = ur.scene_graph.GetMyMutableContextFromRoot(
    diagram_context)
inspector = ur.scene_graph.model_inspector()


## Since the program for the bimanual example is too large for the RAM provided on deepnote, we provide an example region to explore.

In [ ]:
load_file = "dual_ur_regions/dual_ur_binary_search1.npz"

In [ ]:
load_data = np.load(load_file)
C = load_data["C"]
d = load_data["d"]

rational_forward_kin = RationalForwardKinematics(ur.plant)
q_star = np.zeros(ur.plant.num_positions())
ur.plant.SetPositions(plant_context, q_star)
ur.diagram.ForcedPublish(diagram_context)


## We sample many postures to find both the minimum and the maximum distance between the two arms over the region.

In [ ]:
save_posture_file = "dual_ur_closest.npz"
ur3e_demo.sample_closest_posture(ur, rational_forward_kin, q_star,
                       plant_context, C, d, save_posture_file)
saved_posture_data = np.load(save_posture_file)

## We visualize the closest sampled point that we found

In [ ]:
for i in range(2):
    for body_index in ur.plant.GetBodyIndices(
            ur.ur_instances[i]) + ur.plant.GetBodyIndices(
                ur.gripper_instances[i]):
        for body_geometry in inspector.GetGeometries(
                ur.plant.GetBodyFrameIdOrThrow(body_index)):
            if inspector.GetName(
                    body_geometry) != saved_posture_data["min_pair"][i]:
                rgba = np.array([0.5, 0.5, 0.5, 1])
            else:
                rgba = np.array([1., 0, 0, 1])
            SetDiffuse(ur.plant, ur.scene_graph,
                        body_index, inspector.GetName(body_geometry), rgba,
                        scene_graph_context)
ur.plant.SetPositions(plant_context, saved_posture_data["min_q"])
ur.diagram.ForcedPublish(diagram_context)

## Now we visualize the furthest sampled point in the region

In [ ]:
for i in range(2):
    for body_index in ur.plant.GetBodyIndices(
            ur.ur_instances[i]) + ur.plant.GetBodyIndices(
                ur.gripper_instances[i]):
        for body_geometry in inspector.GetGeometries(
                ur.plant.GetBodyFrameIdOrThrow(body_index)):
            if inspector.GetName(
                    body_geometry) != saved_posture_data["max_pair"][i]:
                rgba = np.array([0.5, 0.5, 0.5, 1])
            else:
                rgba = np.array([1., 0, 0, 1])
            SetDiffuse(ur.plant, ur.scene_graph,
                        body_index, inspector.GetName(body_geometry), rgba,
                        scene_graph_context)
ur.plant.SetPositions(plant_context, saved_posture_data["max_q"])
ur.diagram.ForcedPublish(diagram_context)

## Finally, we walk around the generated polytope and highlight the closets collision pair on the two arms.

In [ ]:
walk = viz_utils.generate_walk_around_polytope(HPolyhedron(C,d), 26)

In [ ]:
# Set robot link color. Hightlight the body with the closest distance.
num_samples = 1000
time_points = np.linspace(0, walk.end_time(), num_samples)
for t in time_points:
    s = walk.value(t)
    q = rational_forward_kin.ComputeQValue(s, q_star)
    pair, distance = ur3e_demo.named_closest_distance(ur, plant_context, q, inspector)
    for i in range(2):
        for body_index in ur.plant.GetBodyIndices(
                ur.ur_instances[i]) + ur.plant.GetBodyIndices(
                    ur.gripper_instances[i]):
            for body_geometry in inspector.GetGeometries(
                    ur.plant.GetBodyFrameIdOrThrow(body_index)):
                if inspector.GetName(
                        body_geometry) != inspector.GetName(pair[i]):#saved_posture_data["min_pair"][i]:
                    rgba = np.array([0.5, 0.5, 0.5, 1])
                else:
                    rgba = np.array([1., 0, 0, 1])
                SetDiffuse(ur.plant, ur.scene_graph,
                        body_index, inspector.GetName(body_geometry), rgba,
                        scene_graph_context)
    ur.plant.SetPositions(plant_context, q)
    ur.diagram.ForcedPublish(diagram_context)
    time.sleep(0.1)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7e82e4f5-f47a-475a-aad3-c88093ed36c6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>